<a href="https://colab.research.google.com/github/AUMANSH/Data-Science-Projects/blob/main/HYBRID_SYSTEM_KNN_AND_SVD_WITH_PRINCIPAL_COMPONENTAL_ANALYSIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import shuffle

# Load datasets
auth_data = fetch_lfw_people(min_faces_per_person=70, resize=0.4)
intruder_data = fetch_lfw_people(min_faces_per_person=1, resize=0.4)

# Filter authorized to exclude unknown people
auth_names = set(auth_data.target_names)
intruder_mask = [name not in auth_names for name in intruder_data.target]
X_intruder = intruder_data.data[intruder_mask][:300]
y_intruder = np.zeros(X_intruder.shape[0])

# Intruders
X_auth = auth_data.data
y_auth = np.ones(X_auth.shape[0])

# Combine and shuffle
X_all = np.vstack([X_auth, X_intruder])
y_all = np.hstack([y_auth, y_intruder])
X_all, y_all = shuffle(X_all, y_all, random_state=42)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, stratify=y_all, test_size=0.3, random_state=42)

# PCA via SVD
mean_face = np.mean(X_train, axis=0)
X_train_centered = X_train - mean_face
X_test_centered = X_test - mean_face

U, S, Vt = np.linalg.svd(X_train_centered.T, full_matrices=False)
eigenfaces_k = U[:, :150]

X_train_reduced = np.dot(X_train_centered, eigenfaces_k)
X_test_reduced = np.dot(X_test_centered, eigenfaces_k)

# Train KNN on PCA projections
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train_reduced, y_train)

# Save everything needed for test and evaluation
np.save("mean_face.npy", mean_face)
np.save("eigenfaces_k.npy", eigenfaces_k)
np.save("X_test_reduced.npy", X_test_reduced)
np.save("y_test.npy", y_test)
import pickle
with open("knn_model.pkl", "wb") as f:
    pickle.dump(knn, f)


In [ ]:
import numpy as np
import pickle
from skimage.io import imread
from skimage.transform import resize

# Load saved model and data
mean_face = np.load("mean_face.npy")
eigenfaces_k = np.load("eigenfaces_k.npy")
with open("knn_model.pkl", "rb") as f:
    knn = pickle.load(f)

# Test function
def test_hybrid_knn(image_path):
    img = imread(image_path, as_gray=True)
    img_resized = resize(img, (50, 37), anti_aliasing=True)
    img_vector = img_resized.flatten()
    img_centered = img_vector - mean_face
    img_proj = np.dot(img_centered, eigenfaces_k)

    pred = knn.predict([img_proj])[0]
    result = "Authorized" if pred == 0 else "Intruder"
    print(f"[Hybrid KNN] {result} (Predicted Label: {int(pred)})")
    return pred

# Example usage:
test_hybrid_knn("/content/WhatsApp Image 2025-06-13 at 12.09.24_94781037.jpg")


[Hybrid KNN] Intruder (Predicted Label: 1)


np.float64(1.0)

In [ ]:
import numpy as np
import pickle
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd

# Load saved test set and model
X_test_reduced = np.load("X_test_reduced.npy")
y_test = np.load("y_test.npy")
with open("knn_model.pkl", "rb") as f:
    knn = pickle.load(f)

# Predict and evaluate
y_pred = knn.predict(X_test_reduced)

acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Display in table
df = pd.DataFrame({
    "Metric": ["Accuracy", "Precision", "Recall", "F1-Score"],
    "Hybrid PCA+SVD+KNN": [acc, prec, recall, f1]
})

print(df)


      Metric  Hybrid PCA+SVD+KNN
0   Accuracy            0.813417
1  Precision            0.829646
2     Recall            0.968992
3   F1-Score            0.893921
